In [77]:
import numpy as np
import pprint as pp

In [78]:
def write_yaml_file(file, data):
    with open(file, "w") as f:
        f.writelines(x + '\n' for x in data)

In [79]:
def dict_to_file(options, path):
    data = []
    for opt in options.keys():
        data.append(f"{opt}: {str(options[opt])}")
    
    file_name = f"params_{options['type']}_{options['alphas'][0]}_{options['negative_fix'][0]}_{options['tail_sizes'][0]}-{options['tail_sizes'][-1]}_{options['distance_multipls'][0]}-{options['distance_multipls'][-1]}.yaml"
    write_yaml_file(path + file_name, data)

In [80]:
options = {
    "type": "" ,
    "learning_rate": 0.01,
    "momentum": 0.9,
    "epochs": 1,
    "batch_size": 32,
    "num_clusters_per_class_input": [],
    "num_clusters_per_class_features": [],
    "alphas": "",
    "tail_sizes" : [],
    "distance_multipls": [],
    "negative_fix": "",
    "normalize_factor": "",
    "logger_output": "false",
    "train_only": "false",
    "log_dir": "./logs/",
    "saved_models_dir": "./saved_models/",
    "experiment_data_dir": "./experiment_data/",
    "emnist_dir": "./downloads/",
    "thresholds": [0.0001, 0.001, 0.01, 0.1, 0.3, 0.5, 0.7]
}

In [81]:
types = ["base", "input-cluster", "validation-features-cluster", "training-features-cluster", "input-validation-features-cluster", "input-validation-features-cluster"]
negative_fixes = ["ORIGINAL", "NEGATIVE_VALUE", "VALUE_SHIFT"]
norm = ["NONE", "WEIGHTS", "N-CLASSES", "NORM-WEIGHTS"]
path = "../configs/base/"

## Experiment 1

In [82]:
typ = types[0]
alphas = [-1, 3, 5, 7, 10]
tail_sizes = [10, 25, 50, 75] + np.arange(0, 5100, 100)[1:].tolist()
n = 8
per_file_tail_sizes = [tail_sizes[i:i + n] for i in range(0, len(tail_sizes), n)]
print(per_file_tail_sizes)
dist_multpl = [1.1] + np.arange(1.25, 6, 0.25).tolist()
n = 5
per_file_dist = [dist_multpl[i:i + n] for i in range(0, len(dist_multpl), n)]
print(per_file_dist)

[[10, 25, 50, 75, 100, 200, 300, 400], [500, 600, 700, 800, 900, 1000, 1100, 1200], [1300, 1400, 1500, 1600, 1700, 1800, 1900, 2000], [2100, 2200, 2300, 2400, 2500, 2600, 2700, 2800], [2900, 3000, 3100, 3200, 3300, 3400, 3500, 3600], [3700, 3800, 3900, 4000, 4100, 4200, 4300, 4400], [4500, 4600, 4700, 4800, 4900, 5000]]
[[1.1, 1.25, 1.5, 1.75, 2.0], [2.25, 2.5, 2.75, 3.0, 3.25], [3.5, 3.75, 4.0, 4.25, 4.5], [4.75, 5.0, 5.25, 5.5, 5.75]]


In [83]:
for a in alphas:
    for nf in negative_fixes:
        for no in norm:
            for ts in per_file_tail_sizes:
                for d in per_file_dist:
                    for n in norm:
                        options['type'] = typ
                        options['alphas'] = [a]
                        options['tail_sizes'] = ts
                        options['distance_multipls'] = d
                        options['negative_fix'] = [nf]
                        options['normalize_factor'] = n
                        dict_to_file(options, path)